#使用 Keras 中預定義的典型卷積神經網路結構
tf.keras.applications 中有一些預定義好的典型卷積神經網路結構，如 `VGG16` 、 `VGG19` 、 `ResNet` 、 `MobileNet` 等。

我們可以直接呼叫這些典型的卷積神經網路結構（甚至載入預訓練的參數），而無需手動定義網路結構。

[Keras applications ResNet50 Github](https://github.com/keras-team/keras/blob/6a46d5259d079a58a9d32ad31a9e9da9c0ea563f/keras/applications/resnet.py)

## 引用函式庫

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam , RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

我們可以使用以下代碼來實例化一個 ResNet50 網路結構：

In [1]:
from tensorflow.keras.applications.resnet50 import ResNet50

當執行以上程式碼時，TensorFlow 會自動從網路上下載 ResNet50 網路結構，因此在第一次執行程式碼時需要具備網路連接。

雲端硬碟連接

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 準備資料集

---
[利用 ImageDataGenerator (資料增強) 加強 CNN 辨識率](https://hackmd.io/@allen108108/SyCsOIkxB)



In [6]:
## 定義圖片路徑
trainPath = '/content/drive/MyDrive/Colab Notebooks/江老師/貓兔辨識/train'
validationPath = '/content/drive/MyDrive/Colab Notebooks/江老師/貓兔辨識/valid'
testPath = '/content/drive/MyDrive/Colab Notebooks/江老師/貓兔辨識/test'

## 定義基本參數
imageSize = (300, 300)
imageShape = (300, 300, 3)
numofClass = 2
batchSize = 8
epoch = 20

## 定義 ImageDataGenerator (train)
trainDataGenerator = ImageDataGenerator(rescale = 1 / 255,
                      rotation_range=40, 
                      width_shift_range = 0.3,
                      height_shift_range = 0.3,
                      shear_range = 0.3,
                      zoom_range = 0.2,
                      horizontal_flip = True,
                      fill_mode = 'nearest')
trainBatches = trainDataGenerator.flow_from_directory(directory = trainPath,
                            target_size = imageSize,
                            interpolation = 'bicubic',
                            class_mode = 'categorical',
                            shuffle = True,
                            batch_size = batchSize)

## 定義 ImageDataGenerator (validation)
validateDataGenerator = ImageDataGenerator()
validateBatches = validateDataGenerator.flow_from_directory(directory = validationPath,
                            target_size = imageSize,
                            interpolation = 'bicubic',
                            class_mode = 'categorical',
                            shuffle = False,
                            batch_size = batchSize)

Found 1600 images belonging to 2 classes.
Found 414 images belonging to 2 classes.


In [7]:
for cls, idx in trainBatches.class_indices.items():
    print('Class #{} = {}'.format(idx, cls))

Class #0 = cat
Class #1 = rabbit


## 建立 Transfer Learning 模型
每個網路結構具有自己特定的詳細參數設置，一些共通的常用參數如下：

* input_shape ：輸入張量的形狀（不含第一維的 Batch），大多預設為 224 × 224 × 3 。一般而言，模型對輸入張量的大小有下限，長和寬至少為 32 × 32 或 75 × 75 ；

* include_top ：在網路的最後是否包含全連接層，默認為 True ；

* weights ：預訓練權重值，預設為 'imagenet' ，即為當前模型載入在 ImageNet 資料集上預訓練的權重值。如需隨機初始化變數可設為 None ；

* classes ：分類數，預設為 1000。

In [8]:
myModel = ResNet50(include_top = False,
            weights = 'imagenet',
            input_tensor = None,
            input_shape = imageShape)

x = myModel.output
x = GlobalAveragePooling2D()(x)
x = Dense(units = 512, activation = 'relu')(x)
x = Dropout(0.5)(x)
x = Dense(units = 64, activation = 'relu')(x)
output_layer = Dense(units = numofClass, activation = 'softmax')(x)

myModel.trainable = False
finalModel = Model(inputs = myModel.input, outputs = output_layer)


# 使用 Adam optimizer，以較低的 learning rate 進行 fine-tuning
finalModel.compile(optimizer = Adam(lr=1e-5), loss = 'categorical_crossentropy', metrics = ['accuracy'])


94781440/94765736 [==============================] - 1s 0us/step


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [9]:
finalModel.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 300, 300, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 306, 306, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 150, 150, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [10]:
tf.keras.utils.plot_model(finalModel, show_shapes=True)

Output hidden; open in https://colab.research.google.com to view.

In [11]:
# save the best weights
from keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint(filepath='/content/drive/MyDrive/Colab Notebooks/江老師/貓兔辨識/finalModel.h5', verbose=1, save_best_only=True)

In [12]:
finalModel.fit_generator(trainBatches,
              steps_per_epoch = trainBatches.samples // batchSize,
              validation_data = validateBatches,
              validation_steps = validateBatches.samples // batchSize,
              verbose = 2,callbacks=[checkpointer],
              epochs = epoch)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/20

Epoch 1: val_loss improved from inf to 0.66722, saving model to /content/drive/MyDrive/Colab Notebooks/江老師/貓兔辨識/finalModel.h5
200/200 - 720s - loss: 0.7682 - accuracy: 0.4981 - val_loss: 0.6672 - val_accuracy: 0.5931 - 720s/epoch - 4s/step
Epoch 2/20

Epoch 2: val_loss improved from 0.66722 to 0.66530, saving model to /content/drive/MyDrive/Colab Notebooks/江老師/貓兔辨識/finalModel.h5
200/200 - 57s - loss: 0.7494 - accuracy: 0.4925 - val_loss: 0.6653 - val_accuracy: 0.5956 - 57s/epoch - 283ms/step
Epoch 3/20

Epoch 3: val_loss improved from 0.66530 to 0.65459, saving model to /content/drive/MyDrive/Colab Notebooks/江老師/貓兔辨識/finalModel.h5
200/200 - 57s - loss: 0.7332 - accuracy: 0.5038 - val_loss: 0.6546 - val_accuracy: 0.6201 - 57s/epoch - 285ms/step
Epoch 4/20

Epoch 4: val_loss improved from 0.65459 to 0.65425, saving model to /content/drive/MyDrive/Colab Notebooks/江老師/貓兔辨識/finalModel.h5
200/200 - 57s - loss: 0.7339 - accuracy: 0.4913 - val_loss: 0.6543 - val_accuracy: 0.6127 - 